# Experiments on VQC
### Main experiment

In [9]:
from qiskit import IBMQ, QuantumRegister, QuantumCircuit
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research')

ibmqfactory.load_account:WARNING:2022-05-11 03:54:13,936: Credentials are already in use. The existing account in the session will be replaced.


In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from qiskit.test.mock import FakeJakarta

from qiskit.utils import QuantumInstance
from qiskit.providers.aer.backends import AerSimulator
from qiskit.aqua import aqua_globals
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.circuit.library import RawFeatureVector

import sys
sys.path.append('../../qclib/')
from qclib.machine_learning.bdsp_feature_vector import BdspFeatureVector
from qclib.machine_learning.vqc import VQC
from qclib.machine_learning.datasets import digits

seed = 42

In [3]:
# Dataset load.

sample_total, training_input, test_input, class_labels = digits.load(classes=[0, 1],
                                                                      training_size=40,
                                                                      test_size=10,
                                                                      random_seed=seed)
feature_dim = len(training_input[class_labels[0]][0])

In [13]:
# Parameterized circuit model

n_qubits = int(np.ceil(np.log2(feature_dim)))
var_form = TwoLocal(n_qubits, ['ry', 'rz'], 'cz', reps=1)
var_form.decompose().draw()

┌──────────┐ ┌──────────┐                           ┌───────────┐»
q_0: ┤ Ry(θ[0]) ├─┤ Rz(θ[6]) ├─■──■─────■─────■────────■─┤ Ry(θ[12]) ├»
     ├──────────┤ ├──────────┤ │  │     │     │        │ └───────────┘»
q_1: ┤ Ry(θ[1]) ├─┤ Rz(θ[7]) ├─■──┼──■──┼──■──┼─────■──┼──────────────»
     ├──────────┤ ├──────────┤    │  │  │  │  │     │  │              »
q_2: ┤ Ry(θ[2]) ├─┤ Rz(θ[8]) ├────■──■──┼──┼──┼──■──┼──┼───────■──────»
     ├──────────┤ ├──────────┤          │  │  │  │  │  │       │      »
q_3: ┤ Ry(θ[3]) ├─┤ Rz(θ[9]) ├──────────■──■──┼──■──┼──┼───────┼──────»
     ├──────────┤┌┴──────────┤                │     │  │       │      »
q_4: ┤ Ry(θ[4]) ├┤ Rz(θ[10]) ├────────────────■─────■──┼───────■──────»
     ├──────────┤├───────────┤                         │              »
q_5: ┤ Ry(θ[5]) ├┤ Rz(θ[11]) ├─────────────────────────■──────────────»
     └──────────┘└───────────┘                                        »
«     ┌───────────┐                                                    »
«q_0: ┤ Rz(θ[18]) ├────────────────────────────────────────────────────»
«     └───────────┘┌───────────┐┌───────────┐                          »
«q_1: ──────■──────┤ Ry(θ[13]) ├┤ Rz(θ[19]) ├──────────────────────────»
«           │      └───────────┘└───────────┘┌───────────┐┌───────────┐»
«q_2: ──────┼─────────────────────────■──────┤ Ry(θ[14]) ├┤ Rz(θ[20]) ├»
«           │                         │      └───────────┘├───────────┤»
«q_3: ──────┼────────────■────────────┼────────────■──────┤ Ry(θ[15]) ├»
«           │            │            │            │      └───────────┘»
«q_4: ──────┼────────────■────────────┼────────────┼────────────■──────»
«           │                         │            │            │      »
«q_5: ──────■─────────────────────────■────────────■────────────■──────»
«                                                                      »
«                               
«q_0: ──────────────────────────
«                               
«q_1: ──────────────────────────
«                               
«q_2: ──────────────────────────
«     ┌───────────┐             
«q_3: ┤ Rz(θ[21]) ├─────────────
«     ├───────────┤┌───────────┐
«q_4: ┤ Ry(θ[16]) ├┤ Rz(θ[22]) ├
«     ├───────────┤├───────────┤
«q_5: ┤ Ry(θ[17]) ├┤ Rz(θ[23]) ├
«     └───────────┘└───────────┘

In [5]:
def backend_info(backend):
    backend_name = backend.name()
    backend_qubits = backend.configuration().n_qubits
    print('Experiments using {0} backend, with {1} qubits available.'.format(backend_name, backend_qubits))

def ideal_instance():
    ideal = AerSimulator()
    return QuantumInstance(ideal, shots=1024, seed_simulator=seed, seed_transpiler=seed)

def noisy_instance():
    device = FakeJakarta()
    noisy = AerSimulator.from_backend(device)
    return QuantumInstance(noisy, shots=1024, seed_simulator=seed, seed_transpiler=seed)

def experiment(s, reps, quantum_instance, vqc):
    acc = []
    for i in range(reps):
        print(f's\t{s}\trun\t{i}\t', end='')

        result = vqc.run(quantum_instance)
        acc.append(result["testing_accuracy"])

        print(f'acc\t{result["testing_accuracy"]}')

    print(f's\t{s}\t--\t--\tavg\t{sum(acc) / len(acc)}\tstd\t{np.std(acc)}')

In [14]:
# VQC experiments

# Config.
run_qiskit = False
run_bdsp = True
run_ideal = True
run_noisy = False

reps = 1

bdsp_split = list(range(1, n_qubits+1)))
bdsp_split = [4]
# End config.

aqua_globals.random_seed = seed
algorithm_globals.random_seed = seed

optimizer = SPSA(maxiter=50)
quantum_instances = []
training_size = sum([ len(val) for key, val in training_input.items() ] )
batch_size = int(training_size * 0.1)

if run_ideal:
    quantum_instances.append(ideal_instance())
if run_noisy:
    quantum_instances.append(noisy_instance())

for quantum_instance in quantum_instances:
    backend_info(quantum_instance.backend)

    # Experiment using qiskit initialize function as feature map.
    if run_qiskit:
        print('qiskit')
        qiskit_feature_map = RawFeatureVector(feature_dimension=feature_dim)
        vqc = VQC(optimizer, qiskit_feature_map, var_form, training_input, test_input, None, minibatch_size=batch_size)
        experiment(0.0, reps, quantum_instance, vqc)

    # Experiment using BAA initialize function as feature map.
    if run_bdsp:
        print('bdsp')
        for s in bdsp_split:
            num_qubits = int((s+1)*(feature_dim/(2**s))-1)
            reg = QuantumRegister(num_qubits)
            var_form_2 = QuantumCircuit(reg)
            var_form_2.append(var_form, list(range(var_form.num_qubits)))
            
            bdsp_feature_map = BdspFeatureVector(feature_dimension=feature_dim, split=s)
            
            vqc = VQC(optimizer, bdsp_feature_map, var_form_2, training_input, test_input, None, minibatch_size=batch_size)
            experiment(s, reps, quantum_instance, vqc)

Experiments using aer_simulator backend, with 29 qubits available.
bdsp
circ.num_qubits: 19
s	4	run	0	acc	0.6
s	4	--	--	avg	0.6	std	0.0
